# Oclusão

In [25]:
import numpy as np
import pickle
import json

In [2]:
from dataPreprocess import prepare_data, make_and_save_dict, map_words_and_tags, pad_seq, get_sets

Using TensorFlow backend.


In [9]:
tag2idx = {'I-PER': 0,
           'I-MISC': 1,
           'I-LOC': 2,
           'B-LOC': 3,
           'I-ORG': 4,
           'O': 5,
           'B-ORG': 6,
           'B-MISC': 7,
           'B-PER': 8,
          }
num_tags = len(tag2idx)
tags_set = get_sets(tag2idx)

In [11]:
# import word dict and tag dict
word_dict2 = '../jupyter/model_hu/word_dict2.pkl'
with open(word_dict2, 'rb') as f:
    word2idx = pickle.load(f)
num_words = len(word2idx)
words_set = get_sets(word2idx)
words_set = {}
for wd in word2idx:
    words_set.update({word2idx[wd]: wd})
len(word2idx)

26870

## Import Dataset

In [12]:
# dataset
max_len = 75
num_tags = len(tags_set)
teste = '../jupyter/conll2003/test.txt'
X_te, y_te = prepare_data(teste, num_tags, max_len, word2idx, tag2idx)
treino = '../jupyter/conll2003/train.txt'
X_tr, y_tr = prepare_data(treino, num_tags, max_len, word2idx, tag2idx)

C:\Users\gfra2\anaconda3\envs\projeto_20220828\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


## Import Model

In [13]:
# model
model_glove = '828model_glove.h5'
weights_glove = '../jupyter/model_hu/828model_weights_glove.h5'

from keras.models import load_model
model_no_crf = load_model(model_glove)
model_no_crf.load_weights(weights_glove)

In [14]:
len(model_no_crf.get_weights()),len(model_no_crf.get_weights()[0]),len(model_no_crf.get_weights()[0][0])

(9, 26871, 100)

## LSTM Predict

In [15]:
exp = 307
test_sent = X_te[exp]
p0 = model_no_crf.predict(np.array([test_sent]))
end_sent = np.where(test_sent == 0)[0][0]
pred_matrix = p0[0][:end_sent]

In [16]:
p = np.argmax(p0, axis=-1)
print('Prediction: ')
print(len(p[0][:end_sent]), p[0][:end_sent])

test_true = np.argmax(y_te[exp], -1)
print('True results:')
print(len(test_true[:end_sent]), test_true[:end_sent])

seq_len=0
print()
print("{:2}  {:15}| {:6}  -  {:6}".format("", "Word", "True", "Pred"))
for w, t, pred in zip(X_te[exp], test_true, p[0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}".format(seq_len-1, words_set[w], tags_set[t], tags_set[pred]))

Prediction: 
34 [5 5 8 0 5 5 5 5 8 5 5 5 5 3 5 8 0 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
True results:
34 [5 5 8 0 5 5 5 5 8 5 5 5 5 3 5 8 0 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]

    Word           | True    -  Pred  
 0- tour           | O       -  O     
 1- manager        | O       -  O     
 2- clive          | B-PER   -  B-PER 
 3- lloyd          | I-PER   -  I-PER 
 4- on             | O       -  O     
 5- wednesday      | O       -  O     
 6- apologised     | O       -  O     
 7- for            | O       -  O     
 8- lara           | B-PER   -  B-PER 
 9- 's             | O       -  O     
10- behaviour      | O       -  O     
11- in             | O       -  O     
12- confronting    | O       -  O     
13- australia      | B-LOC   -  B-LOC 
14- coach          | O       -  O     
15- geoff          | B-PER   -  B-PER 
16- marsh          | I-PER   -  I-PER 
17- in             | O       -  O     
18- the            | O       -  O     
19- opposition     | O       -  O     
20- dre

## Occlusion Test

In [18]:
word_idx_target = 2
tag_target = 'B-PER'

print("Predition Shape:", pred_matrix.shape, "\n")
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Predition Shape: (34, 9) 

Alvo
Word: clive          
Tag : B-PER


In [19]:
from ocl_cal import oclusion

In [20]:
sorted_scr_list2 = oclusion (test_sent, tags_set, word_idx_target, tag_target, model_no_crf)

In [21]:
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:18}|  {:18}".format("Sentence", "Ranked Relecance "))
for i, idx in enumerate(sorted_scr_list2.indices):
    print("{:2}- {:15} | {:2}- {:15}".format(i+1, words_set[X_te[exp][i]], i+1, words_set[X_te[exp][idx]]))

Alvo
Word: clive          
Tag : B-PER

  Sentence          |  Ranked Relecance  
 1- tour            |  1- clive          
 2- manager         |  2- lloyd          
 3- clive           |  3- manager        
 4- lloyd           |  4- wednesday      
 5- on              |  5- lara           
 6- wednesday       |  6- apologised     
 7- apologised      |  7- for            
 8- for             |  8- tour           
 9- lara            |  9- confronting    
10- 's              | 10- dressing       
11- behaviour       | 11- geoff          
12- in              | 12- on             
13- confronting     | 13- 's             
14- australia       | 14- coach          
15- coach           | 15- behaviour      
16- geoff           | 16- opposition     
17- marsh           | 17- room           
18- in              | 18- dismissal      
19- the             | 19- his            
20- opposition      | 20- australia      
21- dressing        | 21- in             
22- room            | 22- marsh     

### Occlusion Data Test

In [26]:
data_json1_file = '../jupyter/data.json'
data_json1 = open(data_json1_file)
data1 = json.load(data_json1)
data_json1.close()
print(len(data1['sentences']))
data1

200


{'labels': [{'name': 'B-LOC',
   'description': 'Prineiro nome de uma localidade'},
  {'name': 'B-ORG', 'description': 'Prineiro nome de uma organização'},
  {'name': 'B-PER', 'description': 'Prineiro nome de uma pessoa'},
  {'name': 'I-LOC', 'description': 'Segundo nome de uma localidade'},
  {'name': 'I-ORG', 'description': 'Segundo nome de uma organização'},
  {'name': 'I-PER', 'description': 'Segundo nome de uma pessoa'}],
 'sentences': [[{'id': 10650,
    'sentence': ['(',
     'australia',
     ')',
     ',',
     'mark',
     'roe',
     ',',
     'emanuele',
     'canonica',
     '(',
     'italy',
     ')'],
    'idxAlvo': 1,
    'predictClass': 'B-LOC'}],
  [{'id': 18497,
    'sentence': ['golden', 'state', '6', '13', '.316', '8', '1/2'],
    'idxAlvo': 0,
    'predictClass': 'B-ORG'}],
  [{'id': 8671,
    'sentence': ['soccer',
     '-',
     'thomson',
     'resigns',
     'as',
     'manager',
     'of',
     'raith',
     'rovers',
     '.'],
    'idxAlvo': 2,
    'predic

In [27]:
data_json2_file = '../jupyter/data2.json'
data_json2 = open(data_json2_file)
data2 = json.load(data_json2)
data_json2.close()
print(len(data2['sentences']))
data2

200


{'labels': [{'name': 'B-LOC',
   'description': 'Prineiro nome de uma localidade'},
  {'name': 'B-ORG', 'description': 'Prineiro nome de uma organização'},
  {'name': 'B-PER', 'description': 'Prineiro nome de uma pessoa'},
  {'name': 'I-LOC', 'description': 'Segundo nome de uma localidade'},
  {'name': 'I-ORG', 'description': 'Segundo nome de uma organização'},
  {'name': 'I-PER', 'description': 'Segundo nome de uma pessoa'}],
 'sentences': [[{'id': 10633,
    'sentence': ['2.', 'brian', 'bronson', '(', 'u.s.', ')', '49.67'],
    'idxAlvo': 4,
    'predictClass': 'B-LOC'}],
  [{'id': 178,
    'sentence': ['palestinian',
     'officials',
     'said',
     'the',
     'israeli',
     'government',
     'had',
     'barred',
     'arafat',
     'from',
     'overflying',
     'israel',
     'in',
     'a',
     'palestinian',
     'helicopter',
     'to',
     'the',
     'west',
     'bank',
     'in',
     'an',
     'attempt',
     'to',
     'bar',
     'the',
     'meeting',
     'w

In [28]:
trained_model = model_no_crf

In [30]:
ranked_list = {}
for sent in range(len(data1['sentences'])):
    idxSent      = data1['sentences'][sent][0]['id']
    sentence     = data1['sentences'][sent][0]['sentence']
    tam_sent     = len(sentence)
    idxAlvo      = data1['sentences'][sent][0]['idxAlvo']
    predictClass = data1['sentences'][sent][0]['predictClass']
    idxTag       = tag2idx[predictClass]
    
    if not('' in sentence):
        temp_sent    = np.zeros(75)
        for i, w in enumerate(sentence):
            temp_sent[i] = word2idx[w]
        scores          = trained_model.predict(np.array([temp_sent]))[0]
        pred_matrix     = scores[:tam_sent]
        sorted_scr_list = oclusion(temp_sent, tags_set, idxAlvo, predictClass, trained_model)
        
        relevances = []
        for w in sorted_scr_list[1]:
            relevances.append(sentence[w])
        rel_dict = {'sentence': sentence,
                    'target': sentence[idxAlvo],
                    'class': predictClass,
                    'relevance_ranked': relevances,
                    'ranked_idx': sorted_scr_list[1].tolist()}
        
        ranked_list.update({idxSent: rel_dict})

y = json.dumps(ranked_list)
len(ranked_list)

180

In [ ]:
#jsonFile = open("ocl_ranked1.json", "w")
#jsonFile.write(y)
#jsonFile.close()

In [ ]:
ranked_list = {}
for sent in range(len(data2['sentences'])):
    idxSent      = data2['sentences'][sent][0]['id']
    sentence     = data2['sentences'][sent][0]['sentence']
    tam_sent     = len(sentence)
    idxAlvo      = data2['sentences'][sent][0]['idxAlvo']
    predictClass = data2['sentences'][sent][0]['predictClass']
    idxTag       = tag2idx[predictClass]
    
    if not('' in sentence):
        temp_sent    = np.zeros(75)
        for i, w in enumerate(sentence):
            temp_sent[i] = word2idx[w]
        scores          = trained_model.predict(np.array([temp_sent]))[0]
        pred_matrix     = scores[:tam_sent]
        ocl_scores      = oclusion(temp_sent, trained_model)
            
        ocl_matrix = []
        for i in range(tam_sent):
            olc_sub_matrix = []
            for j in range(tam_sent):
                olc_sub_matrix.append(ocl_scores[i][j])
            ocl_matrix.append(olc_sub_matrix)
        ocl_matrix = np.asarray(ocl_matrix)
        ocl_matrix.shape
    
        delta_scores    = pred_matrix-ocl_matrix[idxAlvo]
        scr_list        = torch.from_numpy(delta_scores.T[idxTag])
        sorted_scr_list = torch.sort(scr_list.abs(), descending =True)
        
        relevances = []
        for w in sorted_scr_list[1]:
            relevances.append(sentence[w])
        rel_dict = {'sentence': sentence,
                    'target': sentence[idxAlvo],
                    'class': predictClass,
                    'relevance_ranked': relevances,
                    'ranked_idx': sorted_scr_list[1].tolist()}
        
        ranked_list.update({idxSent: rel_dict})

y2 = json.dumps(ranked_list)
len(ranked_list)

In [ ]:
#jsonFile = open("ocl_ranked2.json", "w")
#jsonFile.write(y2)
#jsonFile.close()